### Required imports for the following model.

In [ ]:
# For general functionality and model triaing, tuning, testing and visualization
import pandas as pd
import numpy as np
import import_ipynb
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import datasets, layers, models, losses
from sklearn.model_selection import train_test_split
import swifter
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.callbacks import EarlyStopping
import os
from PIL import Image

# For preprocessing
import ipynb
from ipynb.fs.full.capstone_functions import *
import requests
from requests.auth import HTTPBasicAuth
from sentinelhub import MimeType, CRS, BBox, SentinelHubRequest, SentinelHubDownloadClient, \
    DataCollection, bbox_to_dimensions, DownloadRequest, SentinelHubBatch
import datetime
import matplotlib.pyplot as plt
from sentinelhub import SHConfig
from datetime import datetime
import math
import re, ast # Do I need this?
import struct # Do I need this?
import csv
import gdal
import fiona
import rasterio
import rasterio.mask
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error


# 1) Import Code to Preprocess, Extract, and Save Images

#### This section Implements the preprocessing functions created and imported from captsone_funtions.ipynb. For more detailed explanations of the preprocessing pipeline please see capstone_funtions.ipynb.

In [ ]:
# Segment the lights at night tiff down to a specific shape file.
cut_raster_with_shape('/Users/zaneheald/Desktop/Capstone/201912/201912_avg.tif',# Lights at night tiff (for a specific 1/6 qudrant of the globe)
                      '/Users/zaneheald/Desktop/Capstone/bangladesh_1/gadm36_BGD_0.shp', # Shape file of the desired country or region (bangladesh)
                      'bangladesh_raster_final.tif' # Name of the tiff file to save the clipped lights at night tiff to.
                     )

In [ ]:
# Convert the clipped nights at light tiff file to a csv file
data_file = raster_to_csv('/Users/zaneheald/Desktop/Capstone/bangladesh_raster_final')

In [ ]:
# Read in the CSV data extracted above and apply the necesary preporcessing funtions.
data = pd.read_csv(data_file, sep = " ", header = None)
country = pre_process_csv(data)
country_no_outliers = remove_outliers(country,3)
country_agg = agg_radiance(country_no_outliers, 1)
country_boxes = get_boxes(country_agg)

In [ ]:
# Using your unique client id and seceret key from the sentinel hub api interface, apply the image extraction process defined in capstone_funtions.ipynb
client_id = ''
secret = ''

mapped_boxes = map_sentinel_images (country_boxes,client_id, secret)


In [ ]:
# Map images to radiance levels and save these photos to a directory with lat, lon and radiance as the jpeg name.
label_image = map_image_to_labels(country_agg, mapped_boxes)
label_image['image'] = label_image.apply(lambda row: extract_image(row), axis = 1)
save_photos(class_image, 'final_final_final_bangladesh_images')

# 2) Read Images and Implement Final Prepocessing Steps

In [ ]:
# Read in the images, lat, lon, and radiance to a df from a directory.
path = '/Users/zaneheald/Desktop/Capstone/final_final_final_bangladesh_images' #directory containing images (saved above)
data = photo_df(path = path)

### Scale the Radiance Values

In [ ]:
scaler = MinMaxScaler()

data_scaled = data.copy()
data_scaled.Radiance = scaler.fit_transform(np.array(data.Radiance).reshape(-1, 1))

In [ ]:
# Visualize the distribution of Radiance Values
plt.scatter(range(len(data)),data_scaled.Radiance, color = 'Red')
plt.show()

# 3) Prepare Data for Model Creation - TensorFlow Data Set Creation

In [ ]:
def load_and_preprocess(path, label):
    img_height, img_width = 128, 128
    image = tf.io.read_file(path)
    image = tf.image.decode_image(image, channels=3,expand_animations = False)
    image = tf.image.resize(image, [img_height, img_width])
    image /= 255.0
    
    return image, label

In [ ]:
def to_tensor_ds (df,n_epochs,batch_size):
    
    tf.random.set_seed(12345)
    
    ds_files_labels = tf.data.Dataset.from_tensor_slices(
    (df['image_label'],df['Radiance']))
    
    ds_images_labels = ds_files_labels.map(load_and_preprocess) #.shuffle(buffer_size = len(df) - 1)
    
    split_size = int(len(ds_images_labels)*.8) # 80/20 train test split
    
    val_size = int(len(ds_images_labels)*.1) # 10% of train goes to validataion
    ds_train = ds_images_labels.take(split_size)
    ds_test = ds_images_labels.skip(split_size)
    ds_val = ds_train.take(val_size)
    ds_train = ds_train.skip(val_size)
        
    buffer_size = len(ds_train)
    
    ds_train_sh = ds_train.shuffle(buffer_size=buffer_size)
    ds_train_sh = ds_train_sh.batch(batch_size)
    ds_train_sh = ds_train_sh.repeat(n_epochs)
    
    ds_test = ds_test.batch(batch_size= batch_size)
    
    ds_val = ds_val.batch(batch_size = batch_size)
    
    return (ds_train_sh, ds_test, ds_val)
    

In [ ]:
# Set Hyperparameers
n_epochs = 100
batch_size = 512

In [ ]:
ds_train_sh, ds_test, ds_val = to_tensor_ds(data_scaled, n_epochs, batch_size)
steps = np.ceil(len(ds_train_sh) / batch_size)

# 4) Simple CNN

### To compare the value of transfer learning in solving this problem the following CNN has been designed. 

In [ ]:
Input_shape=(128, 128, 3)
cnn_simple=models.Sequential()
cnn_simple.add(layers.Conv2D(16, kernel_size=(5,5), padding="same", activation='relu', input_shape=Input_shape))
cnn_simple.add(layers.MaxPooling2D((2,2)))
cnn_simple.add(layers.Conv2D(32, kernel_size=(3,3), padding="same", activation='relu', input_shape=Input_shape))
cnn_simple.add(layers.MaxPooling2D((4,4)))
cnn_simple.add(layers.Flatten())
cnn_simple.add(layers.Dense(32, activation='relu'))
cnn_simple.add(layers.Dense(1, activation='linear'))

cnn_simple.summary()

cnn_simple.compile(optimizer='adam',
              loss='mean_squared_error',
              metrics=['mae'])

In [ ]:
#Implement Early Stopping to save computational time (as this is a simple model without transfer learning)
es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience = 10)

### Train the simple model:

In [ ]:
%%time
history_simple = cnn_simple.fit(ds_train_sh,
                    epochs=n_epochs,
                    steps_per_epoch = steps,
                    validation_data=ds_val,
                    callbacks=[es],
                    verbose = 1)

### Test the Simple Model:

In [ ]:
x = cnn_simple.evaluate(ds_test, verbose=2)

In [ ]:
pred = cnn_simple.predict(ds_test)

In [ ]:
test_actual = []
for item in ds_test:
    for i in item[1].numpy():
        test_actual.append(i)

mean_squared_error(test_actual,pred)

### Visualize the Simple Model's Outcome (Pred vs Actual, MSE, and Loss Curve) to Evaluate it:

In [ ]:
predicted = plt.scatter(range(len(pred)),pred)
actual = plt.scatter(range(len(pred)),test_actual, color = 'Red', alpha = .08)
plt.legend((predicted,actual),
           ('Predicted', 'Actual_Test_Vale'),
           scatterpoints=1,
           loc='upper right',
           fontsize=8)
plt.title('Predicted vs Actual')
plt.show()

In [ ]:
acc = history_simple.history['mae']
val_acc = history_simple.history['val_mae']

loss = history_simple.history['loss']
val_loss = history_simple.history['val_loss']


plt.figure(figsize=(8, 8))
plt.subplot(2, 1, 1)
plt.plot(acc, label='Training MAE')
plt.plot(val_acc, label='Validation MAE')
plt.legend(loc='upper right')
plt.ylabel('MAE')
plt.ylim([0,.2])
plt.title('Training and Validation MAE')

plt.subplot(2, 1, 2)
plt.plot(loss, label='Training Loss')
plt.plot(val_loss, label='Validation Loss')
plt.legend(loc='upper right')
plt.ylabel('Mean Square Error')
plt.ylim([0,.2])
plt.title('Training and Validation Loss')
plt.xlabel('epoch')
plt.show()

# 5) Transfer Learning

### Set up the requirements / import the transfer learning model. (This implements the Mobile Net Version 2 Model)

In [ ]:
IMG_SIZE = (128, 128)
IMG_SHAPE = IMG_SIZE + (3,)
base_model = tf.keras.applications.MobileNetV2(input_shape=IMG_SHAPE,
                                               include_top=False,
                                               weights='imagenet')
base_model.trainable = False
base_model.summary()

# 6) Model Creation w/ Transfer Learning

### Design a model and implement transfer learning with Mobile Net to improve performance as compared to the simple model. (This model has been iteratively tuned for best performance as measured by MSE):

In [ ]:
layer_1 = layers.Conv2D(32, kernel_size=(5,5), padding="same", activation='relu', input_shape=IMG_SHAPE 
                        ,bias_regularizer=tf.keras.regularizers.l1(0.001))
pool_1 = layers.MaxPooling2D((2,2))
dropout = layers.Dropout(.1, input_shape=IMG_SHAPE)
layer_2 = layers.Conv2D(64, kernel_size=(3,3), padding="same", activation='relu', input_shape=IMG_SHAPE
                       ,bias_regularizer=tf.keras.regularizers.l1(0.01))
pool_2 = layers.MaxPooling2D((2,2))
layer_3 = layers.Conv2D(128, kernel_size=(2,2), padding="same", activation='relu', input_shape=IMG_SHAPE
                       ,bias_regularizer=tf.keras.regularizers.l1(0.01))
pool_3 = layers.MaxPooling2D((1,1))
layer_4 = layers.Conv2D(256, kernel_size=(2,2), padding="same", activation='relu', input_shape=IMG_SHAPE
                       ,bias_regularizer=tf.keras.regularizers.l1(0.01))
pool_4 = layers.MaxPooling2D((1,1))
flatten = layers.Flatten()
dense_1 = layers.Dense(256, activation='relu', bias_regularizer=tf.keras.regularizers.l1(0.01))
prediction_layer = layers.Dense(1, activation='linear')


inputs = tf.keras.Input(shape=(128, 128, 3))
x = base_model(inputs, training=False)
x = layer_1(x)
x = pool_1(x)
x = dropout(x)
x = layer_2(x)
x = pool_2(x)
x = layer_3(x)
x = pool_3(x)
x = layer_4(x)
x = pool_4(x)
x = flatten(x)
x = dense_1(x)
outputs = prediction_layer(x)
cnn_model = tf.keras.Model(inputs, outputs)

In [ ]:
cnn_model.summary()

In [ ]:
cnn_model.compile(optimizer='adam',
              loss='mean_squared_error',
              metrics=['mae'])

In [ ]:
es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience = 20)

### Train the Transfer Learning Model

In [ ]:
%%time
history_sh = cnn_model.fit(ds_train_sh,
                    epochs=n_epochs,
                    steps_per_epoch = steps,
                    validation_data=ds_val,
                    callbacks=[es],
                    verbose = 1)

# 7) Evaluation

### Evaluate the performance of the Transfer Learning Model (numerically and visually) - These metrics were used in interatively tuneing the model.

In [ ]:
x = cnn_model.evaluate(ds_test, verbose=2)

In [ ]:
pred = cnn_model.predict(ds_test)

In [ ]:
data_scaled.Radiance.mean()

In [ ]:
pred.mean()

In [ ]:
pred.min()

In [ ]:
pred.max()

In [ ]:
test_actual = []
for item in ds_test:
    for i in item[1].numpy():
        test_actual.append(i)

mean_squared_error(test_actual,pred)

### Visualize the Predicted vs Actual of the model

In [ ]:
predicted = plt.scatter(range(len(pred)),pred)
plt.title('Predicted vs Actual)')
plt.show()

In [ ]:
predicted = plt.scatter(range(len(pred)),pred)
actual = plt.scatter(range(len(pred)),test_actual, color = 'Red', alpha = .08)
plt.legend((predicted,actual),
           ('Predicted', 'Actual_Test_Vale'),
           scatterpoints=1,
           loc='upper right',
           fontsize=8)
plt.title('Predicted vs Actual (10,000 Data Set Scaled)')
plt.show()

###  Evaluate based on real numbers (rather than scaled numbers)

In [ ]:
pred_real = scaler.inverse_transform(pred.reshape(-1, 1))
test_real = scaler.inverse_transform(np.array(test_actual).reshape(-1, 1))

mean_squared_error(test_real,pred_real)

In [ ]:
predicted = plt.scatter(range(len(pred_real)),pred_real)
plt.title('Predicted vs Actual)')# (10,000 Data Set w/Transfer Learning)')
plt.show()

predicted = plt.scatter(range(len(pred_real)),pred_real)
actual = plt.scatter(range(len(pred_real)),test_real, color = 'Red', alpha = .08)
plt.legend((predicted,actual),
           ('Predicted', 'Actual_Test_Vale'),
           scatterpoints=1,
           loc='upper right',
           fontsize=8)
plt.title('Predicted vs Actual (10,000 Data Set w/Transfer Learning)')
plt.show()

### Evaluate based on urban vs rural areas - The model tends to perform better on rural areas than urban areas:

In [ ]:
# Scaled results:
above_1 = []
above_1_pred = []
below_1 = []
below_1_pred = []
for i, val in enumerate(test_actual):
    if val > .4:
        above_1.append(val)
        above_1_pred.append(pred_ar[i])
    else:
        below_1.append(val)
        below_1_pred.append(pred_ar[i])

        
        
bright_mse = mean_squared_error(above_1,above_1_pred)
print(math.sqrt(bright_mse))

dim_mse = mean_squared_error(below_1,below_1_pred)
print(math.sqrt(dim_mse))

In [ ]:
# Actual unscaled numbers
above_1 = []
above_1_pred = []
below_1 = []
below_1_pred = []
for i, val in enumerate(test_real):
    if val > 2:
        above_1.append(val)
        above_1_pred.append(pred_real[i])
    else:
        below_1.append(val)
        below_1_pred.append(pred_real[i])

# MSE of predictions above radiance of 1
bright_mse = mean_squared_error(above_1,above_1_pred)
math.sqrt(bright_mse)

# MSE of predictions below 1
dim_mse = mean_squared_error(below_1,below_1_pred)
math.sqrt(dim_mse)

### Visualize the MSE and Loss of the model in order to evaluate its utility:

In [ ]:
# Graph the loss and MAE
acc = history_sh.history['mae']
val_acc = history_sh.history['val_mae']

loss = history_sh.history['loss']
val_loss = history_sh.history['val_loss']


plt.figure(figsize=(8, 8))
plt.subplot(2, 1, 1)
plt.plot(acc, label='Training MAE')
plt.plot(val_acc, label='Validation MAE')
plt.legend(loc='upper right')
plt.ylabel('MAE')
plt.ylim([0,.2])
plt.title('Training and Validation MAE')

plt.subplot(2, 1, 2)
plt.plot(loss, label='Training Loss')
plt.plot(val_loss, label='Validation Loss')
plt.legend(loc='upper right')
plt.ylabel('Mean Square Error')
plt.ylim([0,.2])
plt.title('Training and Validation Loss')
plt.xlabel('epoch')
plt.show()

### CNN Creation 
https://towardsdatascience.com/a-guide-to-an-efficient-way-to-build-neural-network-architectures-part-ii-hyper-parameter-42efca01e5d7

https://towardsdatascience.com/ultimate-guide-to-input-shape-and-model-complexity-in-neural-networks-ae665c728f4b

https://www.tensorflow.org/tutorials/images/cnn

https://towardsdatascience.com/guide-to-coding-a-custom-convolutional-neural-network-in-tensorflow-bec694e36ad3

###### Pooling layers
https://machinelearningmastery.com/pooling-layers-for-convolutional-neural-networks/ 

### Transfer learning
https://www.tensorflow.org/tutorials/images/transfer_learning

https://towardsdatascience.com/a-comprehensive-hands-on-guide-to-transfer-learning-with-real-world-applications-in-deep-learning-212bf3b2f27a


### Class Imbalance
https://datascience.stackexchange.com/questions/56447/poor-performance-of-regression-model-for-imbalanced-data

https://www.aaai.org/Papers/Workshops/2000/WS-00-05/WS00-05-001.pdf


### Batch, step, epoch
https://medium.com/@elimu.michael9/understanding-epochs-and-batches-23120a04b3cb

### Data Augmentation
https://www.tensorflow.org/tutorials/images/data_augmentation

### Youtube NN Explained
https://www.youtube.com/watch?v=pHMzNW8Agq4

### l1 vs l2 regularization
https://stats.stackexchange.com/questions/383310/what-is-the-difference-between-kernel-bias-and-activity-regulizers-and-when-t

https://www.machinecurve.com/index.php/2020/01/23/how-to-use-l1-l2-and-elastic-net-regularization-with-keras/

### Fix NaN - Better result ideas
https://stackoverflow.com/questions/37232782/nan-loss-when-training-regression-network

### MSE vs MAE 
https://medium.com/human-in-a-machine-world/mae-and-rmse-which-metric-is-better-e60ac3bde13d

### Adam tuning
https://mlfromscratch.com/optimizers-explained/#momentum

### CNN Cheat Sheet
https://stanford.edu/~shervine/teaching/cs-230/cheatsheet-convolutional-neural-networks